In [4]:
#@title Import Brax and some helper modules
from IPython.display import clear_output

import collections
from datetime import datetime
import functools
import math
import time
from typing import Any, Callable, Dict, Optional, Sequence

try:
	import brax
except ImportError:
	!pip install git+https://github.com/google/brax.git@main
	clear_output()
	import brax

from brax import envs
from brax.envs import to_torch
from brax.io import metrics
from brax.training.agents.ppo import train as ppo
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

# have torch allocate on device first, to prevent JAX from swallowing up all the
# GPU memory. By default JAX will pre-allocate 90% of the available GPU memory:
# https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html
DEVICE = "cpu" if not torch.cuda.is_available() else "cuda"
v = torch.ones(1, device=DEVICE)

In [2]:
DEVICE = "cpu"

In [7]:
gym_name = f'FetchSlide-v2'
if gym_name not in gym.envs.registry.keys():
	entry_point = functools.partial(envs.create_gym_env, env_name=gym_name)
	gym.register(gym_name, entry_point=entry_point)
# env = gym.make(gym_name, batch_size=num_envs, episode_length=episode_length)
env = gym.make(gym_name)
# automatically convert between jax ndarrays and torch tensors:
env = to_torch.JaxToTorchWrapper(env, device=DEVICE)

In [11]:
_ = env.reset()

In [13]:
action = torch.zeros(env.action_space.shape).to(DEVICE)

In [23]:
env.action_space.shape

(4,)

In [10]:
gym_env = JaxToTorchWrapper(env, device=DEVICE)

In [11]:
gym_env

<JaxToTorchWrapper<TimeLimit<OrderEnforcing<PassiveEnvChecker<MujocoFetchSlideEnv<FetchSlide-v2>>>>>>

In [13]:
obs = gym_env.reset()

In [14]:
obs

({'observation': array([ 9.96056349e-01,  7.48908387e-01,  4.12666572e-01,  9.04922569e-01,
          6.83324140e-01,  4.14022562e-01, -9.11337793e-02, -6.55842470e-02,
          1.35598971e-03, -2.03562423e-06,  1.47158977e-03, -5.32964269e-03,
          1.24548654e-04, -1.88202238e-02,  1.22698261e-03, -5.56736934e-05,
          6.43864204e-05,  1.86450993e-02, -4.64845340e-04, -4.83036131e-03,
         -1.26969656e-03,  5.03838696e-05,  7.97511346e-06,  4.66194862e-07,
          5.49241789e-05]),
  'achieved_goal': array([0.90492257, 0.68332414, 0.41402256]),
  'desired_goal': array([1.67190182, 0.88547116, 0.41401894])},
 {})

In [20]:
action = torch.rand(gym_env.action_space.shape, device=DEVICE) * 2 - 1

In [21]:
action

tensor([ 0.1643,  0.0064,  0.0149, -0.5972])

In [22]:
gym_env.step(action)

ValueError: too many values to unpack (expected 4)

In [31]:
sorted(list(gym.envs.registry.keys()))

['Acrobot-v1',
 'AdroitHandDoor-v1',
 'AdroitHandDoorSparse-v1',
 'AdroitHandHammer-v1',
 'AdroitHandHammerSparse-v1',
 'AdroitHandPen-v1',
 'AdroitHandPenSparse-v1',
 'AdroitHandRelocate-v1',
 'AdroitHandRelocateSparse-v1',
 'Ant-v2',
 'Ant-v3',
 'Ant-v4',
 'AntMaze_Large-v3',
 'AntMaze_LargeDense-v3',
 'AntMaze_Large_Diverse_G-v3',
 'AntMaze_Large_Diverse_GDense-v3',
 'AntMaze_Large_Diverse_GR-v3',
 'AntMaze_Large_Diverse_GRDense-v3',
 'AntMaze_Medium-v3',
 'AntMaze_MediumDense-v3',
 'AntMaze_Medium_Diverse_G-v3',
 'AntMaze_Medium_Diverse_GDense-v3',
 'AntMaze_Medium_Diverse_GR-v3',
 'AntMaze_Medium_Diverse_GRDense-v3',
 'AntMaze_Open-v3',
 'AntMaze_OpenDense-v3',
 'AntMaze_Open_Diverse_G-v3',
 'AntMaze_Open_Diverse_GDense-v3',
 'AntMaze_Open_Diverse_GR-v3',
 'AntMaze_Open_Diverse_GRDense-v3',
 'AntMaze_UMaze-v3',
 'AntMaze_UMazeDense-v3',
 'BipedalWalker-v3',
 'BipedalWalkerHardcore-v3',
 'Blackjack-v1',
 'CarRacing-v2',
 'CartPole-v0',
 'CartPole-v1',
 'CartPoleJax-v0',
 'CartPoleJ

In [17]:
obs, reward, done, info = gym_env.step(action)

ValueError: too many values to unpack (expected 4)